In [1]:
%run get_data.ipynb
import simfin as sf
from simfin.names import *
import fredpy as fp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Check if your simfin version is '0.6.0'
print('simfin version:', sf.__version__)

simfin version: 0.8.4


In [21]:
# Set API-key for FRED datasets (free)
fp.api_key = '96aa5f33f8071171250dc08b9033068b'

# Set API-key for Simfin datasets (premium)
sf.set_api_key(api_key="8AGuEXRZYJYssIHmbCps09enKDDOCxLn")

Load financial data and share prices from SimFin

In [22]:
# Set the local directory where data-files are stored.
# The directory wll be created if it does not already exist.
sf.set_data_dir('~/JupyterNotebook/ML Stock Prediction/simfin_data/')

# Then, download the data from the SimFin server and load into a Pandas DataFrame.

In [23]:
df_income_qrt = sf.load_income(variant='quarterly', market='us', refresh_days=1)

Dataset "us-income-quarterly" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!


In [24]:
df_income_qrt

SimFinId Currency  Fiscal Year Fiscal Period Publish Date  \
Ticker Report Date                                                              
A      2010-07-31      45846      USD         2010            Q3   2010-10-06   
       2010-10-31      45846      USD         2010            Q4   2010-12-20   
       2011-01-31      45846      USD         2011            Q1   2011-03-09   
       2011-04-30      45846      USD         2011            Q2   2011-06-07   
       2011-07-31      45846      USD         2011            Q3   2011-09-07   
...                      ...      ...          ...           ...          ...   
ZYNE   2021-06-30     901704      USD         2021            Q2   2021-08-09   
       2021-09-30     901704      USD         2021            Q3   2021-11-15   
       2021-12-31     901704      USD         2021            Q4   2022-03-01   
ZYXI   2021-09-30     171401      USD         2021            Q3   2021-11-02   
       2021-12-31     171401      USD         2021            Q4   2022-03-22   

                   Restated Date  Shares (Basic)  Shares (Diluted)  \
Ticker Report Date                                                   
A      2010-07-31     2011-09-07     347000000.0       352000000.0   
       2010-10-31     2012-12-20     347000000.0       353000000.0   
       2011-01-31     2012-03-05     347000000.0       355000000.0   
       2011-04-30     2012-06-04     347000000.0       355000000.0   
       2011-07-31     2012-09-05     347000000.0       357000000.0   
...                          ...             ...               ...   
ZYNE   2021-06-30     2021-08-09      40065715.0        40065715.0   
       2021-09-30     2021-11-15      40092128.0        40092128.0   
       2021-12-31     2022-03-01      40092128.0        40092128.0   
ZYXI   2021-09-30     2021-11-02      38285500.0        39141300.0   
       2021-12-31     2022-03-22      34167100.0        36325300.0   

                         Revenue  Cost of Revenue  ...  \
Ticker Report Date                                 ...   
A      2010-07-31   1.384000e+09     -659000000.0  ...   
       2010-10-31   1.576000e+09     -742000000.0  ...   
       2011-01-31   1.519000e+09     -703000000.0  ...   
       2011-04-30   1.677000e+09     -777000000.0  ...   
       2011-07-31   1.691000e+09     -799000000.0  ...   
...                          ...              ...  ...   
ZYNE   2021-06-30            NaN              NaN  ...   
       2021-09-30            NaN              NaN  ...   
       2021-12-31            NaN              NaN  ...   
ZYXI   2021-09-30   3.478600e+07       -6837000.0  ...   
       2021-12-31   4.036600e+07       -7331000.0  ...   

                    Non-Operating Income (Loss)  Interest Expense, Net  \
Ticker Report Date                                                       
A      2010-07-31                   -15000000.0            -21000000.0   
       2010-10-31                    35000000.0            -16000000.0   
       2011-01-31                   -13000000.0            -19000000.0   
       2011-04-30                    -6000000.0            -17000000.0   
       2011-07-31                           0.0            -17000000.0   
...                                         ...                    ...   
ZYNE   2021-06-30                     -111585.0                 5943.0   
       2021-09-30                     -371599.0                 5038.0   
       2021-12-31                       21371.0                 4433.0   
ZYXI   2021-09-30                      -18000.0               -18000.0   
       2021-12-31                      -23000.0               -23000.0   

                    Pretax Income (Loss), Adj.  Abnormal Gains (Losses)  \
Ticker Report Date                                                        
A      2010-07-31                    100000000              127000000.0   
       2010-10-31                    238000000                5000000.0   
       2011-01-31                    

In [25]:
df_income_qrt.columns

Index(['SimFinId', 'Currency', 'Fiscal Year', 'Fiscal Period', 'Publish Date',
       'Restated Date', 'Shares (Basic)', 'Shares (Diluted)', 'Revenue',
       'Cost of Revenue', 'Gross Profit', 'Operating Expenses',
       'Selling, General & Administrative', 'Research & Development',
       'Depreciation & Amortization', 'Operating Income (Loss)',
       'Non-Operating Income (Loss)', 'Interest Expense, Net',
       'Pretax Income (Loss), Adj.', 'Abnormal Gains (Losses)',
       'Pretax Income (Loss)', 'Income Tax (Expense) Benefit, Net',
       'Income (Loss) from Continuing Operations',
       'Net Extraordinary Gains (Losses)', 'Net Income',
       'Net Income (Common)'],
      dtype='object')

In [26]:
len(df_income_qrt.reset_index().Ticker.unique())

2547

In [27]:
healthcare_comp = ['A','ABBV','ABC','ABMD','ABT','ALGN','AMGN','ANTM','BAX',
                   'BDX','BIIB','BIO','BMY','BSX','CAH','CERN','CI','CNC','COO',
                   'CRL','CTLT','CVS','DGX','DHR','DVA','DXCM','EW','GILD','HCA',
                   'HOLX','HSIC','HUM','IDXX','ILMN','INCY','IQV','ISRG','JNJ','LH'
                   ,'LLY','MCK','MDT','MOH', 'MRK','MTD','OGN','PFE','PKI','REGN'
                   ,'RMD','STE','SYK','TECH','TFX','TMO','UHS','UNH','VRTX','VTRS'
                   ,'WAT','WST','XRAY','ZBH','ZTS']

In [28]:
len(healthcare_comp)

64

In [29]:
selected_income_qrt=df_income_qrt.reset_index()[df_income_qrt.reset_index().Ticker.isin(healthcare_comp)]

In [30]:
len(selected_income_qrt.Ticker.unique())

61

In [31]:
set(healthcare_comp).difference(selected_income_qrt.Ticker.unique())## they are insurance companies

{'ANTM', 'CI', 'HUM'}

In [32]:
df_balance_qrt = sf.load_balance(variant='quarterly', market='us')

Dataset "us-balance-quarterly" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!


In [33]:
df_balance_qrt

SimFinId Currency  Fiscal Year Fiscal Period Publish Date  \
Ticker Report Date                                                              
A      2010-07-31      45846      USD         2010            Q3   2010-10-06   
       2010-10-31      45846      USD         2010            Q4   2010-12-20   
       2011-01-31      45846      USD         2011            Q1   2011-03-09   
       2011-04-30      45846      USD         2011            Q2   2011-06-07   
       2011-07-31      45846      USD         2011            Q3   2011-09-07   
...                      ...      ...          ...           ...          ...   
ZYNE   2021-06-30     901704      USD         2021            Q2   2021-08-09   
       2021-09-30     901704      USD         2021            Q3   2021-11-15   
       2021-12-31     901704      USD         2021            Q4   2022-03-01   
ZYXI   2021-09-30     171401      USD         2021            Q3   2021-11-02   
       2021-12-31     171401      USD         2021            Q4   2022-03-22   

                   Restated Date  Shares (Basic)  Shares (Diluted)  \
Ticker Report Date                                                   
A      2010-07-31     2010-10-06     347000000.0       352000000.0   
       2010-10-31     2011-12-16     347000000.0       353000000.0   
       2011-01-31     2011-03-09     347000000.0       355000000.0   
       2011-04-30     2011-06-07     347000000.0       355000000.0   
       2011-07-31     2011-09-07     347000000.0       357000000.0   
...                          ...             ...               ...   
ZYNE   2021-06-30     2021-08-09      40065715.0        40065715.0   
       2021-09-30     2021-11-15      40092128.0        40092128.0   
       2021-12-31     2022-03-01      40092128.0        40092128.0   
ZYXI   2021-09-30     2021-11-02      38285500.0        39141300.0   
       2021-12-31     2022-04-28      34167100.0        36325300.0   

                    Cash, Cash Equivalents & Short Term Investments  \
Ticker Report Date                                                    
A      2010-07-31                                      2.317000e+09   
       2010-10-31                                      2.649000e+09   
       2011-01-31                                      2.638000e+09   
       2011-04-30                                      2.975000e+09   
       2011-07-31                                      3.101000e+09   
...                                                             ...   
ZYNE   2021-06-30                                      8.578071e+07   
       2021-09-30                                      7.564250e+07   
       2021-12-31                                      6.780800e+07   
ZYXI   2021-09-30                                      3.536800e+07   
       2021-12-31                                      4.261200e+07   

                    Accounts & Notes Receivable  ...  Short Term Debt  \
Ticker Report Date                               ...                    
A      2010-07-31                   790000000.0  ...     1.501000e+09   
       2010-10-31                   869000000.0  ...     1.501000e+09   
       2011-01-31                   854000000.0  ...     1.000000e+06   
       2011-04-30                   918000000.0  ...     0.000000e+00   
       2011-07-31                   917000000.0  ...     0.000000e+00   
...                                         ...  ...              ...   
ZYNE   2021-06-30                           NaN  ...              NaN   
       2021-09-30                           NaN  ...              NaN   
       2021-12-31                           NaN  ...              NaN   
ZYXI   2021-09-30                    24234000.0  ...              NaN   
       2021-12-31                    28632000.0  ...     5.333000e+06   

                    Total Current Liabilities  Long Term Debt  \
Ticker Report Date                                              
A      2010-07-31                2.917000e+09    2.

In [34]:
df_balance_qrt.columns

Index(['SimFinId', 'Currency', 'Fiscal Year', 'Fiscal Period', 'Publish Date',
       'Restated Date', 'Shares (Basic)', 'Shares (Diluted)',
       'Cash, Cash Equivalents & Short Term Investments',
       'Accounts & Notes Receivable', 'Inventories', 'Total Current Assets',
       'Property, Plant & Equipment, Net',
       'Long Term Investments & Receivables', 'Other Long Term Assets',
       'Total Noncurrent Assets', 'Total Assets', 'Payables & Accruals',
       'Short Term Debt', 'Total Current Liabilities', 'Long Term Debt',
       'Total Noncurrent Liabilities', 'Total Liabilities',
       'Share Capital & Additional Paid-In Capital', 'Treasury Stock',
       'Retained Earnings', 'Total Equity', 'Total Liabilities & Equity'],
      dtype='object')

In [35]:
len(df_balance_qrt.columns)

28

In [36]:
selected_balance_qrt=df_balance_qrt.reset_index()[df_balance_qrt.reset_index().Ticker.isin(healthcare_comp)]

In [37]:
len(selected_balance_qrt.Ticker.unique())

61

In [38]:
df_cashflow_qrt = sf.load_cashflow(variant='quarterly', market='us', refresh_days=30)

Dataset "us-cashflow-quarterly" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!


In [39]:
df_cashflow_qrt.columns

Index(['SimFinId', 'Currency', 'Fiscal Year', 'Fiscal Period', 'Publish Date',
       'Restated Date', 'Shares (Basic)', 'Shares (Diluted)',
       'Net Income/Starting Line', 'Depreciation & Amortization',
       'Non-Cash Items', 'Change in Working Capital',
       'Change in Accounts Receivable', 'Change in Inventories',
       'Change in Accounts Payable', 'Change in Other',
       'Net Cash from Operating Activities',
       'Change in Fixed Assets & Intangibles',
       'Net Change in Long Term Investment',
       'Net Cash from Acquisitions & Divestitures',
       'Net Cash from Investing Activities', 'Dividends Paid',
       'Cash from (Repayment of) Debt', 'Cash from (Repurchase of) Equity',
       'Net Cash from Financing Activities', 'Net Change in Cash'],
      dtype='object')

In [40]:
len(df_cashflow_qrt.columns)

26

In [41]:
selected_cashflow_qrt=df_cashflow_qrt.reset_index()[df_cashflow_qrt.reset_index().Ticker.isin(healthcare_comp)]

In [42]:
len(selected_cashflow_qrt.Ticker.unique())

61

In [43]:
df_info = pd.merge(sf.load_companies(market='us',refresh_days=30),
                   sf.load_industries(refresh_days=30),
                   on = 'IndustryId',
                   how = 'left')

Dataset "us-companies" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
Dataset "industries" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!


In [44]:
df_info.head()

,SimFinId,Company Name,IndustryId,Sector,Industry
0,45846,AGILENT TECHNOLOGIES INC,106001.0,Healthcare,Medical Diagnostics & Research
1,1333027,Li Auto Inc.,NaN,NaN,NaN
2,367153,Alcoa Corp,110004.0,Basic Materials,Metals & Mining
3,939324,"AAC Holdings, Inc.",106011.0,Healthcare,Health Care Providers
4,68568,American Airlines Group Inc.,100006.0,Industrials,Airlines


In [45]:
df_info[df_info.Sector == 'Healthcare']

,SimFinId,Company Name,IndustryId,Sector,Industry
0,45846,AGILENT TECHNOLOGIES INC,106001.0,Healthcare,Medical Diagnostics & Research
3,939324,"AAC Holdings, Inc.",106011.0,Healthcare,Health Care Providers
14,36105,ABAXIS INC,106001.0,Healthcare,Medical Diagnostics & Research
15,61199,AbbVie Inc.,106005.0,Healthcare,Drug Manufacturers
16,187024,AMERISOURCEBERGEN CORP,106014.0,Healthcare,Medical Distribution
...,...,...,...,...,...
3006,1164823,Zomedica Corp.,106001.0,Healthcare,Medical Diagnostics & Research
3008,378251,Zoetis,106005.0,Healthcare,Drug Manufacturers
3012,1243193,Zymergen Inc.,106002.0,Healthcare,Biotechnology
3013,901704,"Zynerba Pharmaceuticals, Inc.",106002.0,Healthcare,Biotechnology


In [49]:
selected_income_qrt[selected_income_qrt['Report Date'] > "2019-12-31"].to_csv("income.csv")

In [50]:
selected_balance_qrt[selected_balance_qrt['Report Date'] > "2019-12-31"].to_csv("balance.csv")

In [51]:
selected_cashflow_qrt[selected_cashflow_qrt['Report Date'] > "2019-12-31"].to_csv("cashflow.csv")